<a href="https://colab.research.google.com/github/GihanMaheshWijesuriya/AngularPOSsytem/blob/productionBranch/SmartyLinkChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [74]:
from keras.models import load_model
model = load_model('model.h5')
import json
import random
intents = json.loads(open('/content/sample_data/smartyLinkBotData.json').read())
words = pickle.load(open('/content/texts.pkl', 'rb'))
classes = pickle.load(open('/content/labels.pkl', 'rb'))

In [75]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)

  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [76]:
def bow(sentence, words, show_details=True):

  sentence_words = clean_up_sentence(sentence)

  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print("found in bag: %s" % w)
  return(np.array(bag))

In [77]:
def predic_class(sentence, model):

  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1], reverse=True)
  result_list = []
  for r in results:
    result_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return result_list

In [85]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if(i['tag'] == tag):
      result = random.choice(i['responses'])
      break
  return result


In [86]:
def chatbot_response(msg):
  ints = predic_class(msg, model)
  res = getResponse(ints, intents)
  return res

In [87]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app) 



In [88]:
@app.route('/')
def home():
    return 'Hello, World!'

In [89]:
@app.route("/get")
def get_bot_response():
  userText = request.args.get('msg')
  return chatbot_response(userText)

In [90]:
if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://34990f785247.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/May/2021 07:08:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2021 07:08:07] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2021 07:08:11] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2021 07:08:28] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2021 07:08:31] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2021 07:08:41] "GET /get?msg=hello HTTP/1.1" 200 -
